<a href="https://colab.research.google.com/github/rudaruda/PythonSFTP/blob/main/Connect_SFTP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testing: FTP/SFTP with Telnet

In [1]:
import socket

def isOpen(ip , port):
  print('...start isOpen(',ip,')...')
  hostname = socket.gethostname()
  local_ip = socket.gethostbyname(hostname)
  print('{Origin:', local_ip, ';', hostname,'}')
  s = socket.socket(socket.AF_INET , socket.SOCK_STREAM)
  print('[set timeout:')
  s.settimeout(5)
  print("'ok']")
  try:
    print('[connect:')
    s.connect((ip , int(port)))
    print("'ok']")
    print('[send]')
    s.sendall(b'Hello, world')
    #s.send("AT\n".encode())
    print("'ok']")
    print('[receive:')
    data = s.recv(1024).decode()
    print("'ok']")
    print('[Recebido:', repr(data),']')
    s.shutdown(2)
    print("{Finish: 'Só Sucesso!'}")
    return True
  except:    
    print("'NOT OK'] ... finish ")
    return False

# Example
#isOpen('12.345.678.901',22)

# Testing: SFTP, connect, check Folder and Files and Write/Read Files

In [ ]:
import paramiko
from stat import S_ISDIR, S_ISREG
from io import BytesIO

def sftp_connect(xHost,xUser,xPass):
  ssh = paramiko.SSHClient()
  # In prod, add explicitly the rsa key of the host instead of using the AutoAddPolicy:
  # ssh.get_host_keys().add('example.com', 'ssh-rsa', paramiko.RSAKey(data=decodebytes(b"""AAAAB3NzaC1yc2EAAAABIwAAAQEA0hV...""")))
  ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
  try:
    ssh.connect( hostname=xHost,username=xUser,password=xPass )
  except Exception as ex: 
      print ("Error on connect SFTP:", ex)
      return -1,-1
  print('[connect: ok]')
  sftp = ssh.open_sftp()
  print('[open: ok]')
  return ssh, sftp
  
def sftp_exists(sftp, path):
  try:
    sftp.stat(path)
    return True
  except FileNotFoundError:
    return False

def sftp_listfolder(sftp, xPath):
  #remotedir = '/' # '/' root dir, '..' parent dir, '.' current dir
  for entry in sftp.listdir_attr(xPath):
    mode = entry.st_mode
    fType = '[Dir.]' if S_ISDIR(mode) else '[File]'
    print(fType, entry.filename)

def sftp_readfile(sftp, xPath):
  try:
    a = sftp.stat(xPath)
  except IOError:
    print('File no exists:', xPath)
    #sftp.put('deleteme.txt', '/tmp/deleteme.txt')
    return -1
  with sftp.open(xPath) as remote_file:
    for line in remote_file:
      print('line:',line)
  print('[read file: ok]')
  
def sftp_writefile(sftp, xPath, xString):
  try:
    a = sftp.stat(xPath)
    print('{ File alread exists:', xPath,'}')
    return -1
  except IOError:
    a=1
    #sftp.put('deleteme.txt', '/tmp/deleteme.txt')
  sftp.putfo(BytesIO(xString.encode()), xPath)
  print('[write file: ok]')
  return 1

def isSftp(xHost,xUser,xPass,xPath='.',xFile=''):
  print('...start isSftp(',xHost,'with user','"{xUser}"',')...')
  xChDir = ''
  if xPath == '': xPath = '.'
  ssh, sftp = sftp_connect(xHost,xUser,xPass)
  if ssh == -1: return -1
  if xPath!='' and xPath not in ['/','.','..']:
      if not sftp_exists(sftp,xPath):
          print ("Error DIR not exists:", xPath)
          return -1
      else:
          print('{dir:',xPath,'}')
          xChDir = xPath
  else:
    stdin, stdout, stderr = ssh.exec_command("pwd")
    print('{home dir:',stdout.readlines(),'}')
  sftp_listfolder(sftp, xPath)
  if xChDir not in ['','.']:
    sftp.chdir(xChDir)
    print('[chdir: ok]')
  if xFile != '': 
    sftp_writefile(sftp, xPath+'/'+xFile, 'Linha 1 Teste Teste Teste\nLinha 2 Testt Testt Testt\nLinha 3 Testi Testi Testi')
  sftp_readfile(sftp,xPath+'/'+xFile)
  sftp.close()

def isSftp2(xHost,xPort,xUser,xPass):
  print('...start isSftp2(',xHost,'with',xUser,')...')
  transport = paramiko.Transport((xHost,xPort))
  try:
    transport.connect(username = xUser, password = xPass)
  except Exception as ex: 
      print ("Error on connect SFTP:", ex)
      return -1
  print('[connect: ok]')
  sftp = transport.open_sftp()
  for filename in sftp.listdir():
    print(f"Downloading {filename} from sftp...")

##Examples
#isOpen('12.345.678.901',22)
#isSftp('12.345.678.901','USER','PASS')
#isSftp('12.345.678.901','USER','PASS','PATH')
#isSftp('12.345.678.901','USER','PASS','PATH','NEWFILE.TXT')

# Alternative

In [ ]:
## FILES TO S3
#mode: ssh treats all files as binary anyway, to 'b' is ignored.
#with sftp.file(filename, mode="r") as file_obj:
#  print(f"uploading  {filename} to s3...")
#  bucket.put_object(Body=file_obj, Key=f"destdir/{filename}")
#  print(f"All done for {filename}")

## Alternative to Write Files [1]
#file=ftp.file('remote file name', "a", -1)
#file.write('Hello World!\n')
#file.flush()
#ftp.close()

## Alternative to Write Files [2]
#from io import BytesIO
#ftp.putfo(BytesIO(localstring.encode()), 'newfile.txt')

## CONNECT WITH FS (FileSystem)
#import fs
#sf = fs.open_fs("sftp://[user[:password]@]host[:port]/[directory]")
#sf.makedir('my_dir')

## Another option to connect with FileSyste
#from fs.sshfs import SSHFS
#sf = SSHFS(...

## Another another option to connect with FileSyste
#from fsspec.implementations.sftp import SFTPFileSystem
#fs = SFTPFileSystem(host=host, username=username, password=password)
#fs.ls("/")
#with fs.open(file_name) as file:
#  content = file.read()